In [1]:
from librep.datasets.har.loaders import (
    MotionSense_BalancedView20HZ
)
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT
import numpy as np

ModuleNotFoundError: No module named 'librep'

In [2]:
X = [[0], [3], [1], [45], [28], [24], [35], [14], [67], [52], [98]]
import numpy as np
X = np.random.rand(12000,2000)*10000000
print(X.shape)
from sklearn.neighbors import kneighbors_graph
W = kneighbors_graph(X, 10, mode='distance',
                                metric='minkowski', p=2, metric_params=None, 
                                include_self=False, n_jobs=-1)
# A.toarray()


(12000, 2000)


In [8]:
import pandas as pd
dictio = {"titulo": [3,6,9,7,3]}
df = pd.DataFrame(dictio)
print(df)
ejem = df.get(0)
print(ejem)

   titulo
0       3
1       6
2       9
3       7
4       3
None


In [27]:
pd.Series([1,2,4], index = ["x", "y", "z"])


x    1
y    2
z    4
dtype: int64

In [34]:
from numpy import random
random.rand()
np.fix(23.99)
arr = np.array([1,2,3,4,5,6,7])


# print(arr[::2])
# random.normal(size=1000, loc=50, scale=0.2)
# arr.shape
arr = np.array([1,2,3])
arr = np.array([1, 2, 3, 4], dtype='f')
arr = np.array([1, 2, 3, 4]).toFloat()


AttributeError: 'numpy.ndarray' object has no attribute 'toFloat'

In [3]:
from scipy import sparse, linalg

num_FPS = 500
Numpts = W.shape[0]
indices_FPS = [];
indices_FPS.append(np.random.randint(0,Numpts))
    
Dsk = np.zeros([num_FPS,Numpts])
         
for itern in range(0,num_FPS-1):
        # first compute the distances:
    Dsk[itern,:] = sparse.csgraph.dijkstra(W,directed = False,indices=indices_FPS[itern])
        # print(itern, Dsk)
    indices_FPS.append(np.argmax(np.ndarray.min(Dsk[0:itern+1,:],0)))
        # print(itern, indices_FPS)
Dsk[itern+1,:] = sparse.csgraph.dijkstra(W,directed = False,indices=indices_FPS[-1])
    
Landmarks = {'indices':indices_FPS,'Ds': Dsk,'W':W}
Dsk

array([[5.22552336e+08, 5.22474707e+08, 5.23276071e+08, ...,
        5.25971072e+08, 6.96818675e+08, 5.24592446e+08],
       [6.97211262e+08, 6.98540212e+08, 5.27462709e+08, ...,
        5.28869895e+08, 6.98995062e+08, 6.99335569e+08],
       [5.27389814e+08, 5.30265271e+08, 7.01483310e+08, ...,
        5.26554476e+08, 5.26975924e+08, 7.01272895e+08],
       ...,
       [5.23882600e+08, 6.97004463e+08, 6.94720316e+08, ...,
        5.25262717e+08, 6.96188289e+08, 6.94832464e+08],
       [5.23511903e+08, 6.98344117e+08, 6.96725677e+08, ...,
        5.24124644e+08, 5.24648707e+08, 6.96837826e+08],
       [6.93543048e+08, 5.23339350e+08, 6.92509963e+08, ...,
        6.94988744e+08, 6.96093879e+08, 6.92622112e+08]])

In [6]:
Dsk.max()

873074146.5433558

In [ ]:
loader = MotionSense_BalancedView20HZ(
    root_dir="data/views/MotionSense/balanced_view_20Hz_with_gravity_9.81_acc_standard", 
    download=False
)
# Load the dataset
train_val, test = loader.load(concat_train_validation=True, label=loader.standard_label)
train_val, test

In [ ]:
train_HD = np.array(train_val[:][0])
train_LD = None
train_Y = np.array(train_val[:][1])
test_HD = np.array(test[:][0])
test_LD = None
test_Y = np.array(test[:][1])

# Aplicar FFT

In [ ]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(
    transforms=[fft_transform],
    new_window_name_prefix="fft."
)

In [ ]:
train_dataset_fft = transformer(train_val)
test_dataset_fft = transformer(test)

In [ ]:
train_HD = train_dataset_fft.X
train_LD = None
train_Y = train_dataset_fft.y
test_HD = test_dataset_fft.X
test_LD = None
test_Y = test_dataset_fft.y

In [ ]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

# DIMAL

In [ ]:
import torch
import myFunctions
import sklearn.neighbors as nb
import Functions_ManifoldLearning as FnManifold
import MDSNet_3D


In [ ]:
D = test_HD

In [ ]:
torch.manual_seed(1000)

myFunctions.tic()
# Compute the Graph
W = nb.kneighbors_graph(D, 10, mode='distance',
                        metric='minkowski', p=2, metric_params=None, 
                        include_self=False, n_jobs=-1)
Landmarks = FnManifold.FPS(W,500)
myFunctions.toc()

In [ ]:
W
assert 1==0

In [ ]:
numIndices = 200
NetParams = {'Size_HL':70,'Num_HL':2}
LearningParams = {'Numiter':2000,'learning_rate':1e-2, 'tol':1e-3}

Landmarks_FPS = {'indices':Landmarks['indices'][0:numIndices],
                     'Ds':Landmarks['Ds'][0:numIndices]}

In [ ]:
#==============================================================================
#                       RunMDS
#==============================================================================
Embedding_MDSNet,model,train_err  = MDSNet_3D.MDSNet(D,Landmarks_FPS,NetParams,LearningParams)
myFunctions.toc()

In [ ]:
test_HD_input = torch.from_numpy(test_HD).float()
print(type(test_HD_input), test_HD_input)

In [ ]:
test_LD = model(test_HD_input).detach().numpy()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
def visualize(X, Y):
    labels = ['sit', 'stand', 'walk', 'stair up', 'stair down', 'run']
    df = pd.DataFrame(dict(x=X[:,0], y=X[:,1], label=Y))
    groups = df.groupby('label')
    fig, ax = plt.subplots()
    ax.margins(0.05)
    for name, group in groups:
        ax.plot(group.x, group.y, marker='.', linestyle='', ms=8, label=labels[name])
    ax.legend()
    plt.show()

In [ ]:
visualize(test_LD, test_Y)

In [ ]:
# Visualizing test
tsne = TSNE()
test_2D = tsne.fit_transform(test_LD)
visualize(test_2D, test_Y)

In [ ]:
print(test_LD)

In [ ]:
assert 1==0

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 10 11:53:47 2017

@author: gipadmin
"""

from IPython import get_ipython
get_ipython().magic('reset -sf') 

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import myFunctions
from sklearn.neighbors import NearestNeighbors
import sklearn.neighbors as nb
from sklearn import manifold
from sklearn import preprocessing
from scipy import sparse, linalg
from matplotlib import cm
import MDSNet_3D
import Functions_ManifoldLearning as FnManifold
from sklearn import preprocessing, manifold, datasets
import torch
import scipy.io as sio

torch.manual_seed(1000)
#==============================================================================
#                   Generate The Data
#==============================================================================
Numpts = np.power(2,11)
#D, color = datasets.samples_generator.make_s_curve(n_samples=Numpts, noise=0.0, random_state=1)
D, color = datasets.make_s_curve(n_samples=Numpts, random_state=0)

index = np.random.permutation(np.arange(D.shape[0]))
D = D[index,:]
color = color[index]

myFunctions.tic()
# Compute the Graph
W = nb.kneighbors_graph(D, 10, mode='distance',
                              metric='minkowski', p=2, metric_params=None, 
                              include_self=False, n_jobs=-1)
Landmarks = FnManifold.FPS(W,500)
myFunctions.toc()

numIndices = 200
NetParams = {'Size_HL':70,'Num_HL':2}
LearningParams = {'Numiter':2000,'learning_rate':1e-2, 'tol':1e-3}

Landmarks_FPS = {'indices':Landmarks['indices'][0:numIndices],
                     'Ds':Landmarks['Ds'][0:numIndices]}
    
#==============================================================================
#                       RunMDS
#==============================================================================
Embedding_MDSNet,Net,train_err  = MDSNet_3D.MDSNet(D,Landmarks_FPS,NetParams,LearningParams)
myFunctions.toc()
# print(Embedding_MDSNet.shape)
#==============================================================================
#                       Plot
#==============================================================================
plt.close('all')
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.set_aspect('equal')
ax1.scatter(Embedding_MDSNet[:,0],Embedding_MDSNet[:,1], c = color,s=20,lw=0,alpha=1,cmap=cm.jet)

# ax1.scatter(Embedding_MDSNet[Landmarks_FPS['indices'],0],Embedding_MDSNet[Landmarks_FPS['indices'],1], c = 'k',s=20,lw=0,alpha=1,cmap=cm.jet)
ax1.scatter(Embedding_MDSNet[Landmarks_FPS['indices'],0],
            Embedding_MDSNet[Landmarks_FPS['indices'],1],s=20,lw=0,alpha=1)

ax1.set_title(str(numIndices),fontsize = 10)

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 10 11:53:47 2017

@author: gipadmin
"""

from IPython import get_ipython
get_ipython().magic('reset -sf') 

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import myFunctions
from sklearn.neighbors import NearestNeighbors
import sklearn.neighbors as nb
from sklearn import manifold
from sklearn import preprocessing
from scipy import sparse, linalg
from matplotlib import cm
import MDSNet_3D
import Functions_ManifoldLearning as FnManifold
from sklearn import preprocessing, manifold, datasets
import torch
import scipy.io as sio

torch.manual_seed(1000)
#==============================================================================
#                   Generate The Data
#==============================================================================
Numpts = np.power(2,11)
#D, color = datasets.samples_generator.make_s_curve(n_samples=Numpts, noise=0.0, random_state=1)
D, color = datasets.make_s_curve(n_samples=Numpts, random_state=0)

index = np.random.permutation(np.arange(D.shape[0]))
D = D[index,:]
color = color[index]

myFunctions.tic()
# Compute the Graph
W = nb.kneighbors_graph(D, 10, mode='distance',
                              metric='minkowski', p=2, metric_params=None, 
                              include_self=False, n_jobs=-1)
Landmarks = FnManifold.FPS(W,500)
myFunctions.toc()

numIndices = 200
NetParams = {'Size_HL':70,'Num_HL':2}
LearningParams = {'Numiter':2000,'learning_rate':1e-2, 'tol':1e-3}

Landmarks_FPS = {'indices':Landmarks['indices'][0:numIndices],
                     'Ds':Landmarks['Ds'][0:numIndices]}
    
#==============================================================================
#                       RunMDS
#==============================================================================
Embedding_MDSNet,Net,train_err  = MDSNet_3D.MDSNet(D,Landmarks_FPS,NetParams,LearningParams)
myFunctions.toc()
# print(Embedding_MDSNet.shape)
#==============================================================================
#                       Plot
#==============================================================================
plt.close('all')
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.set_aspect('equal')
ax1.scatter(Embedding_MDSNet[:,0],Embedding_MDSNet[:,1], c = color,s=20,lw=0,alpha=1,cmap=cm.jet)

# ax1.scatter(Embedding_MDSNet[Landmarks_FPS['indices'],0],Embedding_MDSNet[Landmarks_FPS['indices'],1], c = 'k',s=20,lw=0,alpha=1,cmap=cm.jet)
ax1.scatter(Embedding_MDSNet[Landmarks_FPS['indices'],0],
            Embedding_MDSNet[Landmarks_FPS['indices'],1],s=20,lw=0,alpha=1)

ax1.set_title(str(numIndices),fontsize = 10)

In [ ]:
[key for key in Landmarks]

In [ ]:
print('indices-len', len(Landmarks['indices']))
print('Ds-shape', Landmarks['Ds'].shape)
print('W', Landmarks['W'].shape)

In [ ]:
print(D.shape)
print(min(D[0,:]), max(D[0,:]))
print(min(D[1,:]), max(D[1,:]))
print(min(D[2,:]), max(D[2,:]))

In [ ]:
color

In [ ]:
index = np.random.permutation(np.arange(D.shape[0]))
print(np.random.permutation(np.arange(10)))

In [ ]:
X = [[0, 0], [3, 0], [1, 0]]
from sklearn.neighbors import kneighbors_graph
A = kneighbors_graph(X, 1, mode='distance', include_self=False)
A.toarray()